**_Hypothesis_**: The popularity of a category depends on the publishing date

   - **Metric**: Popularity: average review/score

In [14]:
# Import libraries

import pymongo
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
# Connect to MongoDB

client = pymongo.MongoClient('mongodb://localhost:27017/')
database = client['spark_db']
books = database['books_joined']

In [16]:
# Define the aggregation pipeline
pipeline = [
        {
        "$project": {
            'review/score': 1,
        'categories': 1,
        'publishedDate': 1,
        }
    },
    {'$match': {
        'review/score': {'$exists': True, '$ne': 0.0},
        'categories': {'$exists': True, '$ne': None},
        'publishedDate': {'$exists': True, '$ne': None},
    }
    }
]

# Execute the aggregation query and create a DataFrame
source_df = pd.DataFrame(list(books.aggregate(pipeline)))

In [25]:
source_df[source_df['publishedDate'].__contains__(
    '-')] = pd.to_datetime(source_df[source_df['publishedDate'].__contains__('-')]).dt.year
source_df

KeyError: False